##1. 데이터 준비

In [1]:
from google.colab import drive
import glob
import os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
txt_file_path = '/content/drive/MyDrive/Colab Notebooks/lyricist/lyrics/*'

In [4]:
txt_list = glob.glob(txt_file_path)

In [5]:
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Build your dreams to the stars above', 'But when you need someone to love', "Don't go to strangers, darling, come to me Play with fire till your fingers burn"]




---



##2.데이터 정제

소스 문장(Source Sentence) 및 타겟 문장(Target Sentence)준비

In [6]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿,),()]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        AAA $$ sen-tence."))

<start> this is sample aaa sen tence . <end>


필터링 후 정제된 문장 확인

In [7]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(preprocess_sentence(sentence).split()) > 15: continue
        
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> build your dreams to the stars above <end>',
 '<start> but when you need someone to love <end>',
 '<start> and when there s no place for you to turn <end>',
 '<start> you ll follow your heart i know <end>',
 '<start> i ve been through it all , for i m an old hand <end>',
 '<start> and i ll understand if you go so <end>',
 '<start> make your mark for your friends to see <end>',
 '<start> but when you need more than company <end>',
 '<start> you give your soul so sweetly <end>',
 '<start> tonight the light of love is in your eyes <end>']



---



##3.정제된 데이터 토큰화 (데이터 전처리)

tf.keras.preprocessing.text.Tokenizer 패키지

: 정제된 데이터를 토큰화, 단어 사전(vocabulary 또는 dictionary라고 칭함)생성, 데이터를 숫자로 변환

In [8]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   

    # 토큰의 개수가 15개 초과하는 문장을 학습데이터에서 제외
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 1366   19 ...    0    0    0]
 [   2   33   45 ...    0    0    0]
 [   2    8   45 ...    0    0    0]
 ...
 [   2  234    1 ...    0    0    0]
 [   2   10  500 ...    0    0    0]
 [   2  123   19 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fcd8a993b50>


단어 사전 확인

In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to




---



##4.소스와 타겟 분리

In [10]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 1366   19  388   10    6  623  678    3    0    0    0    0    0]
[1366   19  388   10    6  623  678    3    0    0    0    0    0    0]


학습데이터 개수를 124960 미만으로 설정하기 위해서, 2.데이터정제 과정에서 가사 중 괄호 ), ( 를 특수문자와 함께 제거하였음.

In [11]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=15)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Test:", enc_val.shape)
print("Target Test:", enc_val.shape)

Source Train: (124735, 14)
Target Train: (124735, 14)
Source Test: (31184, 14)
Target Test: (31184, 14)




--- 임의 테스트



In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>



---



##5.인공지능 학습

인공지능 모델 생성

embedding_size와 hidden_size를 변경하면서 2.2수준의 val_loss값 도출

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

모델 학습

1. embedding_size = 256, hidden_size = 2048 ==> val_loss = 2.20

2. embedding_size = 256, hidden_size = 4096 ==> val_loss = 2.24

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
488/488 [==============================] - 959s 2s/step - loss: 3.4152 - val_loss: 3.0103
Epoch 2/10
488/488 [==============================] - 877s 2s/step - loss: 2.8811 - val_loss: 2.8098
Epoch 3/10
488/488 [==============================] - 864s 2s/step - loss: 2.6316 - val_loss: 2.6631
Epoch 4/10
488/488 [==============================] - 866s 2s/step - loss: 2.3968 - val_loss: 2.5450
Epoch 5/10
488/488 [==============================] - 879s 2s/step - loss: 2.1584 - val_loss: 2.4446
Epoch 6/10
488/488 [==============================] - 877s 2s/step - loss: 1.9192 - val_loss: 2.3576
Epoch 7/10
488/488 [==============================] - 877s 2s/step - loss: 1.6882 - val_loss: 2.3000
Epoch 8/10
488/488 [==============================] - 864s 2s/step - loss: 1.4794 - val_loss: 2.2553
Epoch 9/10
488/488 [==============================] - 881s 2s/step - loss: 1.3035 - val_loss: 2.2366
Epoch 10/10
488/488 [==============================] - 878s 2s/step - loss: 1.1711 - val_lo



---



##6. 문장 생성 

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측하지 않았거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you, much more than you'



---
## 회고




*   원하는 loss 값이 나오게 하기 위해 여러 수치들을 변경하며 테스트를 진행한다. 이때, 앞으로는 처음 값부터 최종 변경 값까지 정확히 기록을 하며 진행해야겠다. 변경하면서 loss 값이 더 안 좋아지는 경우에는 이전 값으로 돌아가야하는 상황이 발생할 수 있다는 것을 염두에 두어야 한다.



*   val_loss 값을 2.2 이하로 낮추기 위하여 시도한 것

> 1. embedding_size 변경 : 256 --> 14 --> 256  (maxlen과 동일하게 14를 넣었을 때, loss 값은 더 악화 됨.)


> 2. hidden_size 변경 : 1024 --> 2048 --> 3072 --> 4096 --> 2048



*   hidden_size 값이 증가할수록 학습 시간이 길어졌다.
*   코랩에서 실행 시 hidden_size 값을 4096으로 했을 때, 학습시간이 거의 2시간이 걸렸다.
*   hidden_size 값을 2048로 했을 때 결과값이 가장 좋았기 때문에, 다시 값을 변경하여 학습시켰다.
*   물리적 시간의 한계로 LMS 클라우드 주피터노트북으로 코랩과 듀얼로 학습을 시켰다.
*   Epoch 9 학습을 하고 있는데, 커널 연결이 끊겨서 1시간이 날라갔다.
*   colab GPU가 hidden_size 4096으로 학습한 뒤 GPU 사용이 제한되었다.
*   다음부터는 특정 값을 변경하여 테스트할 때, colab을 구글 계정 여러개로 동시에 돌려야겠다.
*   loss 값을 낮추기 위해, 혹은 적당한 loss 값을 위해 설정해야 하는 수치에 대한 설명이 노드에서 부족한 듯 하다. 구글링을 통해 따로 학습을 해야겠다.

